### 손글씨 이미지 데이터 MNIST

- 인코딩된 바이너리 데이터를 디코딩하여 처리하는 방식 확인
- 지도 학습
- 학습용 데이터는 6만개, 테스트 데이터는 1만개
- 결론
    - 학습 후 새로운 데이터 입력시 판별
    - 0~9까지의 손글씨 이미지를 판별
    - 데이터는 url을 직접 획득해서, 원하는 곳에 다운로드

|  No  | 단계                         | 내용                                                         |
| :--: | :--------------------------- | :----------------------------------------------------------- |
|  1   | 연구목표                     | - 손글씨 이미지(0~9)를 학습시켜서 새로운 손글씨 이미지를 판별하는 머신러닝 모델 구축<br>- 이미지 압축 해제<br>- 데이터 디코딩<br>- 28x28로 구성된 픽셀 이미지 벡터화<br/>- 시스템 통합의 결과를 보고 연구 목표 설정 필요하지만 시스템 통합 생략 |
|  2   | 데이터획득/수집              | - http://yann.lecun.com/exdb/mnist/ 접속<br>- Web Scraping을 통해 데이터의 URL 획득<br/>- 지정된 위치에 다운로드 -> 압축해제 |
|  3   | 데이터준비/전처리            | - 디코딩(내부구조를 알 수 있는 인코딩 문서(MNIST Database) 필요)<br>- 에디언 처리<br>- 벡터화 |
|  4   | 데이터탐색/통찰/시각화분석   | - skip                                                       |
|  5   | 데이터모델링(머신러닝모델링) | - 분류 알고리즘 사용<br>- 알고리즘 선정- > 학습/테스트데이터 -> 학습 -> 예측 -> 성능평가 |
|  6   | 시스템통합                   | - skip                                                       |

#### 2. 데이터 획득/수집
- 모듈 준비

In [38]:
import urllib.request as req
from bs4 import BeautifulSoup

- web scraping

In [39]:
rootUrl = 'http://yann.lecun.com/exdb/mnist/'
soup = BeautifulSoup(req.urlopen(rootUrl), 'html5lib')

- train-images-idx3-ubyte.gz, ... 등 총 4개 url 획득

In [40]:
files = [ tt.a.string for tt in soup.findAll('tt')[:4]]
files

['train-images-idx3-ubyte.gz',
 'train-labels-idx1-ubyte.gz',
 't10k-images-idx3-ubyte.gz',
 't10k-labels-idx1-ubyte.gz']

- 다운로드 > 압축해제 

In [41]:
import os, os.path, gzip

In [42]:
# 파일 다운로드 위치 선정
savePath = '../data/mnist'
if not os.path.exists(savePath):
    os.makedirs(savePath)

In [43]:
# 진행률 모듈
from tqdm import tqdm_notebook
for file in tqdm_notebook(files):
    print('소스', rootUrl+file)
    
    # 저장 위치 및 파일명
    local_path = f'{savePath}/{file}'
    print('대상', local_path)
    
    # 웹상에 존재하는 리소스를 로컬 디스크에 직접 저장
    req.urlretrieve(rootUrl+file, local_path)

소스 http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
대상 ../data/mnist/train-images-idx3-ubyte.gz
소스 http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
대상 ../data/mnist/train-labels-idx1-ubyte.gz
소스 http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
대상 ../data/mnist/t10k-images-idx3-ubyte.gz
소스 http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
대상 ../data/mnist/t10k-labels-idx1-ubyte.gz



In [45]:
# 압축해제
for file in tqdm_notebook(files):
    # 원본 파일의 경로
    ori_gzip_file = f'{savePath}/{file}'
    # 압축해제 파일의 경로
    target_file = f'{savePath}/{file[:-3]}'
    # 압축해제
    with gzip.open(ori_gzip_file, 'rb') as fg:
        tmp = fg.read()
        with open(target_file, 'wb') as f:
            f.write(tmp)

### 3. 데이터 준비/전처리 

- 디코딩(내부구조를 알 수 있는 인코딩 문서(MNIST Database) 필요)
- 에디언(Endian) 처리(TCP/IP 상에서 통신 수행시 중요)
    - 컴퓨터 메모리와 같은 1차원 공간에 여러 개의 연속된 데이터를 배열하는 방법
    - 종류: 바이트를 배치하는 순서에 따라
        - 빅 에디언: 값을 앞에서부터 채움(ex. 0x12 0x34 0x56 0x78)
        - 리틀 에디언: 값을 뒤에서부터 채움(ex. 0x78 0x56 0x34 0x12)
- 벡터화

- LABEL FILE
    - magic number: 4byte -> 에디안 체크
    - LABEL 수: 4byte -> 에디안 체크
    - LABEL 데이터: 1byte -> 0~9
    - 크기: 4 + 4 + LABEL 수 * 1byte = 8 + 60000 = 60008byte
- IMAGE FILE
    - magic number: 4byte -> 에디안 체크
    - 손글씨 이미지 개수: 4byte -> 에디안 체크
    - 가로크기(픽셀수) : 4byte -> 에디안 체크
    - 세로크기(픽셀수) : 4byte -> 에디안 체크
    - 픽셀값 : unsigned 1byte -> 0~2^8-1: 0~255(0xFF)

In [46]:
# 바이너리 데이터를 빅/리틀 에디안 방식으로 특정 바이트만큼 읽는 기능 제공
import struct

In [69]:
# 헤더 정보 추출
label_f = open('../data/mnist/t10k-labels-idx1-ubyte','rb')
image_f = open('../data/mnist/t10k-images-idx3-ubyte','rb')
# 바이너리 데이터는 헤더부터 읽어서 데이터의 유효성이나 종류를 인지함
# MNIST 규격서: high edian(>)으로 기술돼있고, label은 헤더가 4+4=8byte이다
label_magic_number, label_count = struct.unpack('>II',label_f.read(4+4))
image_magic_number, image_count, row, col = struct.unpack('>IIII',image_f.read(4+4+4+4))

print(label_magic_number, label_count)
print(image_magic_number, image_count, row, col)
print('이미지 파일의 크기:', 4+4+4+4+image_count*row*col,'bytes')

2049 10000
2051 10000 28 28
이미지 파일의 크기: 7840016 bytes


In [70]:
csv_f = open('../data/mnist/t10k.csv', 'w', encoding='utf-8')
pixels = row * col

for idx in range(image_count):
    # 정답 추출
    label_tmp = struct.unpack('B', label_f.read(1))
    label = label_tmp[0]
    
    # 이미지 추출
    binaryData = image_f.read(pixels)
    strData = list(map(lambda x : str(x), binaryData))
    csv_f.write(str(label)+',')
    csv_f.write(','.join(strData)+'\n')
    
    # pgm 파일로 dump(데이터 유효성 확인 위함)
    with open(f'../data/mnist/{label}.pgm','w',encoding='utf-8') as f:
        f.write('P2 28 28 255\n' + ' '.join(strData))
    # 이미지 데이터의 벡터화
    break

In [71]:
label_f.close()
image_f.close()
csv_f.close()

In [74]:
def decoding_mnist_rawData(dataStyle='train', maxCount=0):
    label_f = open(f'../data/mnist/{dataStyle}-labels-idx1-ubyte','rb')
    image_f = open(f'../data/mnist/{dataStyle}-images-idx3-ubyte','rb')
    csv_f = open(f'../data/mnist/{dataStyle}.csv', 'w', encoding='utf-8')
    label_magic_number, label_count = struct.unpack('>II',label_f.read(4+4))
    image_magic_number, image_count, row, col = struct.unpack('>IIII',image_f.read(4+4+4+4))

    pixels = row * col

    for idx in tqdm_notebook(range(image_count)):
        if idx >= maxCount: break
        label_tmp = struct.unpack('B', label_f.read(1))
        label = label_tmp[0]

        binaryData = image_f.read(pixels)
        strData = list(map(lambda x : str(x), binaryData))
        csv_f.write(str(label)+',')
        csv_f.write(','.join(strData)+'\n')

        with open(f'../data/mnist/{label}.pgm','w',encoding='utf-8') as f:
            f.write('P2 28 28 255\n' + ' '.join(strData))
        
    label_f.close()
    image_f.close()
    csv_f.close()
decoding_mnist_rawData('t10k',2500)
decoding_mnist_rawData('train',7500)